In [2]:
import pandas as pd
data = pd.read_csv("../data/linkedin_posts.csv")
data2 = pd.read_csv("../data/hate_speech.csv")
data3 = pd.read_csv("../data/postings.csv")

length1 = len(data)

In [3]:
data = data.iloc[:716]

#Append data2['text'] to data['Post Content']
job_posting = pd.DataFrame({'Post Content': data3['description'].tolist()}).sample(frac=1, random_state=42).reset_index(drop=True).iloc[:500]
data = pd.concat([data, job_posting], ignore_index=True)

# Append data2['text'] to data['Post Content']
hate_speech = pd.DataFrame({'Post Content': data2['text'].tolist()}).sample(frac=1, random_state=42).reset_index(drop=True).iloc[:500]
data = pd.concat([data, hate_speech], ignore_index=True)

# Convert all post contents to lowercase
data['Post Content'] = [text.lower() for text in data['Post Content']]
# Remove URLs
data['Post Content'] = data['Post Content'].str.replace(r'\bhttps?://\S+\b', '', regex=True)
# Remove punctuation (periods, commas, question marks, double quotes, etc.)
data['Post Content'] = data['Post Content'].str.replace(r'[^a-zA-Z ]', '', regex=True)

post_content = data['Post Content'].tolist()
job_des = data3['description'].tolist()



In [4]:
import re

# Save post_content to a file called posts.txt
with open("posts.txt", "w", encoding="utf-8") as f:
    i = 0
    for post in post_content:
        i += 1
        f.write(str(i) + ". " + post + "\n")

In [5]:
# Read classified.txt and append its content as a new column to the data dataframe
with open("classified.txt", "r", encoding="utf-8") as f_classified:
    classified_labels = [line.strip() for line in f_classified]
    
length2 = len(classified_labels)

for i in range(length2):
    if re.search(r'\bnot\b', str(classified_labels[i]), re.IGNORECASE):
        classified_labels[i] = 0 #not cringe
    else:
        classified_labels[i] = 1 #cringe

print(len(classified_labels)) #should be 716
    
classified_labels.extend([0] * len(job_posting)) #for the job postings
    
classified_labels.extend([1] * len(hate_speech)) #for the hate speech posts

print(len(classified_labels))


716
1716


In [6]:
final_data = data

final_data['Classified'] = classified_labels

final_data = final_data.sample(frac=1, random_state=42).reset_index(drop=True)

test_data = final_data.iloc[len(final_data)//2:].copy()
final_data = final_data.iloc[:len(final_data)//2]

In [7]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences=final_data["Post Content"], vector_size=100, window=5, min_count=2, workers=4, sg = 1) #sg = 1 for skip gram
word2vec_model.save("word2vec.model") #save to file named word2vec.model

In [ ]:
import tensorflow as tf
keras = tf.keras
models = tf.keras.models
layers = tf.keras.layers
tokenizer_module = tf.keras.preprocessing.text
sequence_module = tf.keras.preprocessing.sequence
Tokenizer = tokenizer_module.Tokenizer
pad_sequences = sequence_module.pad_sequences
import numpy as np

# --- 1. Define Model Hyperparameters (YOU NEED TO SET MAX_SEQUENCE_LENGTH) ---
# This is the maximum number of words/tokens in any input post.
MAX_SEQUENCE_LENGTH = 150 # Example: assuming most posts are <= 150 words
EMBEDDING_DIM = 100       # This comes from your Word2Vec model's vector_size

# Number of classes for binary classification (Good/Bad)
NUM_CLASSES = 1 # For binary classification, output a single probability

# --- 2. Build the CNN Model ---
def build_text_cnn_model(max_seq_length, embedding_dim, num_classes):
    model = models.Sequential()

    # Layer 1: Conv1D (Convolutional Layer for text)
    # filters: Number of feature detectors/filters to learn (e.g., to detect n-gram patterns)
    # kernel_size: The size of the sliding window (e.g., 5 words at a time)
    # activation: ReLU for non-linearity
    # input_shape: (sequence_length, embedding_dimension)
    model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu',
                            input_shape=(max_seq_length, embedding_dim)))

    # Layer 2: MaxPooling1D (Pooling Layer for text)
    # pool_size: The size of the pooling window. Reduces sequence length.
    model.add(layers.MaxPooling1D(pool_size=2))
    # This will reduce the length of the sequence by half.

    # Optional:
    # model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    # model.add(layers.MaxPooling1D(pool_size=2))

    # Layer 3: Flatten Layer
    # Converts the 2D output from MaxPooling1D (length x filters) into a 1D vector
    # This prepares the data for the fully connected (Dense) layers.
    model.add(layers.Flatten())

    # Layer 4: Dense (Fully Connected) Hidden Layer
    # units: Number of neurons in this layer.
    # activation: ReLU for non-linearity.
    model.add(layers.Dense(units=64, activation='relu'))
    # You can experiment with the number of units here.

    # Layer 5: Dense (Output Layer)
    # units: 1 for binary classification (predicting a probability).
    # activation: 'sigmoid' to output a probability between 0 and 1.
    model.add(layers.Dense(units=num_classes, activation='sigmoid'))

    return model

# Create the model instance
model = build_text_cnn_model(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, NUM_CLASSES)

# Print a summary of the model's architecture
model.summary()

# --- 3. Compile the Model ---
# optimizer: How the model updates its weights during training. Adam is a good default.
# loss: Binary Crossentropy for binary classification problems.
# metrics: What to monitor during training (e.g., accuracy).
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



2025-06-30 03:44:00.196408: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-30 03:44:00.199604: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-30 03:44:00.208532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751255040.218648   14511 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751255040.221702   14511 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751255040.231333   14511 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 146, 128)       │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 73, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9344)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       598,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 662,273 (2.53 MB)

 Trainable params: 662,273 (2.53 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# --- 4. Prepare Your Data (Conceptual Steps - you'll need to implement this) ---
from sklearn.model_selection import train_test_split

# 1. Tokenize your posts (convert text to lists of words)
# Example (using Keras's Tokenizer for simplicity, but you might use custom tokenization):
tokenizer = Tokenizer()
tokenizer.fit_on_texts(final_data['Post Content'].tolist()) # Fit tokenizer on your post content
word_index = tokenizer.word_index # Vocabulary mapping word to ID

# 2. Convert posts to sequences of word IDs (using your tokenizer)
sequences = tokenizer.texts_to_sequences(final_data['Post Content'].tolist())

# 3. Pad/Truncate sequences to MAX_SEQUENCE_LENGTH
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# 4. Convert padded sequences of word IDs into sequences of Word2Vec vectors
# This is the crucial step to get the input shape (None, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)
X = np.zeros((len(padded_sequences), MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
for i, sequence in enumerate(padded_sequences):
    for j, word_id in enumerate(sequence):
        word = tokenizer.index_word.get(word_id) # Convert ID back to word
        if word in word2vec_model.wv: # Check if word is in Word2Vec vocabulary
            X[i, j] = word2vec_model.wv[word]
        else:
            # Handle out-of-vocabulary words (e.g., keep as zeros or use a special UNK vector)
            pass
y = np.array(final_data['Classified'].tolist()) # Convert labels to numpy array

# --- 5. Split Data (Training, Validation) ---
# This code splits your data into training and validation sets, but does not create a separate test set.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 6. Train the Model (Conceptual) ---
model.fit(X_train, y_train,
            epochs=10, # Number of training iterations
            batch_size=32, # Number of samples per gradient update
            validation_data=(X_val, y_val)) # Data to evaluate on after each epoch

# --- 7. Evaluate the Model (Conceptual) ---
loss, accuracy = model.evaluate(X_train, y_train) ##fix to x and y test later!!
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/10


2025-06-30 03:44:04.049021: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 41160000 exceeds 10% of free system memory.


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5304 - loss: 0.6692 - val_accuracy: 0.6919 - val_loss: 0.5601
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7051 - loss: 0.5732 - val_accuracy: 0.7558 - val_loss: 0.5789
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7691 - loss: 0.5037 - val_accuracy: 0.7442 - val_loss: 0.5814
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8006 - loss: 0.4316 - val_accuracy: 0.7558 - val_loss: 0.5753
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8085 - loss: 0.3794 - val_accuracy: 0.7791 - val_loss: 0.5895
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8370 - loss: 0.3615 - val_accuracy: 0.8023 - val_loss: 0.6385
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8606 - loss: 0.3195 - val_accuracy: 0.7733 - val_loss: 0.6698
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8544 - loss: 0.2847 - val_accuracy: 0.8023 - val_loss: 0.

2025-06-30 03:44:08.967694: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 41160000 exceeds 10% of free system memory.


In [ ]:
# Preprocess test_data["Post Content"] to match model input
test_sequences = tokenizer.texts_to_sequences(test_data["Post Content"].tolist())
test_padded = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_test = np.zeros((len(test_padded), MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
for i, sequence in enumerate(test_padded):
	for j, word_id in enumerate(sequence):
		word = tokenizer.index_word.get(word_id)
		if word in word2vec_model.wv:
			X_test[i, j] = word2vec_model.wv[word]

y_test = np.array(test_data["Classified"].tolist())

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 72.14%


2025-06-30 03:45:47.770026: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51480000 exceeds 10% of free system memory.


In [ ]:

# --- 8. Make Predictions (Conceptual) ---
new_post_text = ""
#Try taking a random post from the test_data that is classified as "not bad" (0)
for i in range(len(test_data)):
    if test_data['Classified'].iloc[i] == 0:
        new_post_text = test_data['Post Content'].iloc[i]
        break
    
print(new_post_text)

# Preprocess new_post_text into a single Word2Vec sequence (X_new)
sequence = tokenizer.texts_to_sequences([new_post_text])
padded = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_new = np.zeros((1, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
for j, word_id in enumerate(padded[0]):
    word = tokenizer.index_word.get(word_id)
    if word in word2vec_model.wv:
        X_new[0, j] = word2vec_model.wv[word]

prediction_proba = model.predict(X_new)[0][0]
if prediction_proba > 0.5: # Threshold for "bad"
    print("This post is classified as BAD.")
else:
    print("This post is classified as GOOD.")

one month into my ms marketing program a researchintensive degree at iba i found myself turning to chatgpt for validation of my english language skills everyweek i am assigned multiple readings in all four of my courses to complete before the next session and while i am not someone who enjoys reading particularly i have been putting in consistent effort to do what is expected of me academic readings however are a completely different ballgame i often find myself reading a sentence multiple times yet still struggling to understand its meaning it feels as if english has betrayed methough the words seem familiar the sentence as a whole doesnt make sense ai tools like chatgpt and copilot ai have been incredibly helpful in clarifying difficult passages my current approach is to first read a simplified aigenerated summary of an article or paper which helps me grasp the general idea before diving into the full text if anyone has helpful tips for an art student looking to improve her reading s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
This post is classified as GOOD.


In [ ]:
# --- 8. Make Predictions (Conceptual) ---
new_post_text = ""
#Try taking a random post from the test_data that is classified as "bad" (1)
for i in range(len(test_data)):
    if test_data['Classified'].iloc[i] == 1:
        new_post_text = test_data['Post Content'].iloc[i]
        break
    
print(new_post_text)

# Preprocess new_post_text into a single Word2Vec sequence (X_new)
sequence = tokenizer.texts_to_sequences([new_post_text])
padded = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_new = np.zeros((1, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
for j, word_id in enumerate(padded[0]):
    word = tokenizer.index_word.get(word_id)
    if word in word2vec_model.wv:
        X_new[0, j] = word2vec_model.wv[word]

prediction_proba = model.predict(X_new)[0][0]
if prediction_proba > 0.5: # Threshold for "bad"
    print("This post is classified as BAD.")
else:
    print("This post is classified as GOOD.")

stop saying that refugees are vile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
This post is classified as BAD.
